### Keras нь өндөр түвшний мэдрэлийн сүлжээний API бөгөөд хэрэглэгчидэд ээлтэй байдал, хурдан загварчилал, модульчилагдсан байдал, өрөгтгөх чөдварт төвлөрдөг. Энэ нь Tensorflow, Theano, CNTK зэрэг гүнзгий сургалтын систимуудтэй ажилууладаг тул бид нейрон сүлжээг асуудалгүй сургах боломжтой.

### MNIST нь гараар бичсэн цифрүүдийн 70,000 зургийг агуулдаг: 60,000 нь сургалтанд, 10,000 нь
туршилтанд зориулагдсан. Зургууд нь саарал өнгөтэй, 28x28 пикселтэй бөгөөд урьдчилан
боловсруулалтыг багасгаж, хурдан эхлүүлэхийн тулд төвлөрсөн байна. Бид хэдэн арван
мянган гараар бичсэн зургийн мэдээллийн сангаас цифрүүдийг зөв тодорхойлохыг зорьж

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import dataset
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
transform = torchvision.transforms.Compose([
    torchvision.trabsforms.RandomRotation(10),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.1307,),(0.3081,))
])
mnist_train = torchvision.datasets.MNIST('./data', train = True, download = True, transform = )

mnist_test = torchvision.datasets.MNIST('./data', train = False, download = True, transform = )

In [ ]:
mnist_train

In [ ]:
len(mnist_train)

In [ ]:
len(mnist_test)

In [ ]:
image = mnist_train.data[0]

In [ ]:
image.shape

In [ ]:
image[0][0]

In [ ]:
plt.imshow(image)
plt.title(mnist_train.targets[0])
plt.show()

Том өгөгдлийн багцтай ажиллахын тулд тэдгээрийг бүгдийг нэг дор санах ойд ачаалах
шаардлагатай. Ихэнх тохиолдолд бид системд байгаа санах ойн хэмжээ хязгаарлагдмал тул
санах ойн тасалдалтай тулгардаг. Мөн нэг удаа ачаалагдсан хүнд дата багцаас болж
програмууд удаан ажиллах хандлагатай байдаг. PyTorch нь DataLoader ашиглан өгөгдөл
ачаалах процессыг batch size шийдлийг санал болгодог. Өгөгдлийн ачааллыг параллель
болгохын тулд Dataloader ашигласан бөгөөд энэ нь хурдыг нэмэгдүүлж, санах ойг хэмнэдэг.
Өгөгдөл ачаалагч бүтээгч нь torch.utils.data багцад байрладаг. Энэ нь янз бүрийн
параметрүүдтэй бөгөөд тэдгээрийн хооронд дамжуулагдах ёстой цорын ганц аргумент нь
ачаалагдах ёстой өгөгдлийн багц бөгөөд бусад нь бүгд нэмэлт аргументууд юм.
Синтакс: DataLoader(dataset, shuffle=True, sampler=None, batch_size=32)
Энд датасетийг batch size 1000 ширхэгээр хувааж оруулж байна.

In [ ]:
train_loader = DataLoader(mnist_train, batch_size = 1000, shuffle = True)
test_loader = DataLoader(mnist_test, batch_size = 1000, shuffle = True)

In [ ]:
for x, y in train_loader:
    print(x.shape, y.shape)
    print(x[0][0][0][0], y[0])
    break

### Convolutional Neural Network үүсгэж байна.

max_pool2d: Макс pooling нь дээж дээр суурилсан ялгах үйл явц юм. Зорилго нь оролтын
дүрслэлийг (зураг, далд давхаргын гаралтын матриц гэх мэт) доош түүвэрлэн, хэмжээсийг
нь багасгаж, давхардсан дэд бүсүүдэд агуулагдах онцлогуудын талаар таамаглал дэвшүүлэх
боломжийг олгох явдал юм.


In [ ]:
class Net(nn.Module):
    def __init__(self, hidden_feature_size = 100):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size = 5) # features extraction
        self.conv2 = nn.Conv2d(10, 20, kernel_size = 5) # features extraction
        self.fc1 = nn.Linear(320, 100)
        self.fc2 = nn.Linear(hidden_feature_size, 10)
        self.dp = nn.Dropout2d()
        self.relu = nn.ReLU()

    def forward(self, x):
        x = nn.functional.max_pool2d(self.dp(self.relu(self.conv1(x))),2) # nn.functional gej
        x = nn.functional.max_pool2d(self.dp(self.relu(self.conv2(x))),2)
        x = x.view(-1, 320)
        x = self.relu(self.fc1(x))
        x = nn.functional.log_softmax(self.fc2(x), dim = -1)
        return x

dummy_data = torch.rand(10, 1, 28, 28)

In [ ]:
dummy_data.shape

In [ ]:
net = Net()

In [ ]:
out = net(dummy_data)
out.shape

In [ ]:
out

1. loss.backward()-н дараа optimizer.step()-г дуудаж өгөх хэрэгтэй байдаг учраас
zero_grad()-г дууддаг. Бүр тодруулбал, алдагдал.backward() болон optimizer.step() гэсэн
хоёр үйлдлийг тусгаарласан ба optimizer.step() нь зөвхөн тооцоолсон градиент
шаарддаг тул градиент автоматаар тэглэгддэггүй.

2. Loss function-г урьдчилан таамагласан үр дүн болон хүссэн үр дүн хоорондох алдааг
хэмжихэд ашигладаг. Loss function нь хүсэж буй үр дүнгээс хэр хол байгааг хэлж өгдөг.

3. loss.backward() # бүх параметрийн градиент олдог буюу backpropagation юм.
Backpropagation нь хүссэн үр дүн болон гарсан үр дүнгийн хоорондох ялгааг буюу
гарсан алдааг багасгахад оршдог. Энэ нь градиент функцийн доошлох чиглэлийн дагуу
алдааг тооцоолж явдаг ба хамгийн доор байгаа утга нь хамгийн бага утга байдаг.
Дээрээс доошоо тоооцоолдог учир backward гэж бас нэрлэдэг.

4. optimizer.step() # бид хамгийн бага алдагдал (алдаа) гаргахын тулд эдгээр
параметрүүдийг шинэчилдэг.

5. Cross entropy loss алдагдал нь нь 0-ээс 1 хүртэлх тоогоор хэмжигддэг бөгөөд 0 нь төгс
загвар юм. Гол зорилго нь загварыг аль болох 0-д ойртуулах явдал юм. Урьдчилан
таамагласан магадлал нь бодит тэмдэглэгээнээс зөрөх тусам cross entropy loss нь
нэмэгддэг.

6. torch.optim.SGD: Implements stochastic gradient descent (optionally with momentum).
Optimizer нь neural network-н жин, суралцах хурд зэрэг шинж чанаруудыг өөрчилдөг
функц эсвэл алгоритм юм. Тиймээс энэ нь нийт log loss бууруулж, нарийвчлалыг
сайжруулахад тусалдаг.

In [ ]:
def train(net, train_loader, test_loader, loss_fn, optimizer, epochs):
    net.train()
    accurancy = torch.tensor([0.])
    for x, y in train_loader:
        optimizer.zero_grad()
        preds = net(x)
        print(preds.shape)
        loss = loss_fn(preds, y)
        loss.backward() # backprogation, update hiideg
        optimizer.step()
        print(net.fc2.weight.grad[0][0])
        print(net.fc1.weight.grad[0][0])
        print(net.conv1.weight.grad[0][0])
        print(net.conv2.weight.grad[0][0])
        print(loss.item())
        break

loss_fn = nn.CrossEntropyLoss()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr = 0.001, momentum = 0.5)
epochs = 1
train(net, train_loader, test_loader, loss_fn, optimizer, epochs)

In [ ]:
def train(net, train_loader, test_loader, loss_fn, optimizer, epochs):
    net.train()
    for e in range(epochs):
        accuracy = torch.tensor([0.])
        for x, y in train_loader:
            optimizer.zero_grad()
            preds = net(x)
            loss = loss_fn(preds, y)
            loss.backward() #backpropagation, update hiideg
            optimizer.step()
            preds = preds.data.max(1, keepdim = True)[1]
            accuracy += preds.eq(y.data.view_as(preds)).sum()
        print('epochs:', e, 'Accuracy:', accuracy.item()*100/len(mnist_train))
        eval(net, test_loader)

def eval(net, test_loader):
        net.eval()
        accuracy = torch.tensor([0])
        for x, y in test_loader:
            preds = net(x)
            preds = preds.data.max(1, keepdim = True)[1]
            accuracy += preds.eq(y.data.view_as(preds)).sum()
        print("test acc:", accuracy.item()*100/len(mnist_test))

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr = 0.001, momentum = 0.5)
epochs = 2
train(net, train_loader, test_loader, loss_fn, optimizer, epochs)

In [ ]:
torch.save(net.state_dict(), './checkpoints') # model ee hadaglana

In [ ]:
del net # model-ee ustgana

In [ ]:
cnn = Net() # shineeer model - ee duudaj ugnu

In [ ]:
train(cnn, train_loader, test_loader, loss_fn, optimizer, epochs)

In [ ]:
cnn.load_state_dict(torch.load('./checkpoints')) # hadgalsan model -ee load hiine

In [ ]:
train(cnn, train_loader, test_loader, loss_fn, optimizer, epochs)